# Notebook for testing ls agent API both locally and remotely

In [ ]:
import requests
import json
import os
import jwt
import openai

In [ ]:
openai.api_key  = os.environ['OPENAI_API_KEY']

client = openai.OpenAI()


In [ ]:
# ! pip install certifi

In [ ]:
import certifi

## Make sure the environmental variables setup to appropriate values
After that restart the jupyter notebook server

In [ ]:
ls_app_url =  'http://pocketverse.herokuapp.com/LS_API'
ls_app_username = os.environ['LS_USERNAME']
ls_app_password = os.environ['LS_PASSWORD']
ls_app_username, ls_app_password

In [ ]:
ls_agent_url_local = 'http://127.0.0.1:5000'
ls_agent_ulr_remote = 'https://ls-agent-cd78b4dee2fb.herokuapp.com'
BASE_URL = ls_agent_ulr_remote
# BASE_URL = ls_agent_url_local
print (f'Testing ls agent on {BASE_URL}')

## Test utils and basic tests

In [ ]:
import base64
import json

def decode_jwt(token):
    header, payload, signature = token.split('.')
    header_decoded = base64.urlsafe_b64decode(add_padding(header)).decode('utf-8')
    payload_decoded = base64.urlsafe_b64decode(add_padding(payload)).decode('utf-8')

    return {
        "header": json.loads(header_decoded),
        "payload": json.loads(payload_decoded),
        "signature": signature
    }

def add_padding(str):
    """Adds padding to the Base64 encoded string to make it valid."""
    return str + '=' * (4 - len(str) % 4)

In [ ]:
def test_ls_get (route, url = BASE_URL, token = ''):
    """ Function to access unprotected and protected route using the JWT token. """
    url = f'{url}/{route}'
    headers = {'Authorization': f'Bearer {token}'}
    rsp = requests.get(url, headers = headers, verify=certifi.where())
    print(rsp.status_code)
    try:
        print(json.dumps(rsp.json()))
    except Exception as e:
        print(str(e))
    return


In [ ]:
test_ls_get('')

In [ ]:
def get_access_token(username, password, login, url = BASE_URL):
    """ Function to authenticate and receive access token from the server. """
    url_r = f"{url}/{login}"
    js =  {"username": username, 
           "password": password, 
           "reportLoginResult": "true"}
    print (url_r)
    response = requests.post(url_r,json = js)
    if response.status_code == 200:
        return response.json()
    else:
        print (response.status_code, response.json())
        raise Exception("Failed to authenticate")

In [ ]:
resp_agent = get_access_token(ls_app_username, ls_app_password, 'login')
resp_agent 

In [ ]:
resp_app = get_access_token(ls_app_username, ls_app_password, 'jwt_login', url = ls_app_url)
resp_app

In [ ]:
def test_ls_post (data_out: dict, route: str, url = BASE_URL, token = '')-> str:
    headers = {
    "Content-Type": "application/json",
    'Authorization': f'Bearer {token}'
    }
    url = f'{url}/{route}'
    response = requests.post(url, json=data_out, headers=headers)
    if response.status_code == 200:
        data = response.json()
        answer = data.get('answer', 'No answer received')
        return f"Answer: {answer}"
    else:
        print (response.status_code, response.json())
        raise Exception("Failed to authenticate")
    return
    


In [ ]:

def access_route(token, route, url = BASE_URL):
    """ Function to access a protected route using the JWT token. """
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(f"{BASE_URL}/{route}", headers=headers)
    return response.text


## Use pocket app token to access protected route on ls-agent

In [ ]:
token_pocket = resp_app['token']
access_route(token_pocket, 'protected')

## Acess protected route on ls-agent with dummy token. Should fail

In [ ]:
access_route('blabla.fff.gg', 'protected')

## Tests

In [ ]:
test_ls_get('')

In [ ]:
test_ls_get('protected', token = token_pocket)

### Make sure openai is on

In [ ]:
data_out = {"question":"Why is the sky blue? Answer in 15 words"}
question = data_out['question']
print (f'Question: {question}')
response = client.chat.completions.create(
                model="gpt-3.5-turbo-0613",
                temperature = 0,
                messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": question}
                        ]
                )
answer = response.choices[0].message.content
print (f'Answer: {answer}')

### Testing unprotected route

In [ ]:
data_out = {"question":"What's your name?"}
answer = test_ls_post(data_out,'openai-direct')
print (f'Answer: {answer}')

### Testing protected route without a token. It should fail

In [ ]:
data_out = {"question":"What is Python language? Answer in 15 words"}
test_ls_post(data_out,'openai-direct-protected')

### Testing the same with the token. Should work

In [ ]:
test_ls_post(data_out,'openai-direct-protected', token = token_pocket)